In [1]:
# all the necessary imports
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch import optim
import torch
import torchtext
from torchtext.data import Field, LabelField
from torchtext.data import TabularDataset
from torchtext.data import Iterator, BucketIterator

In [2]:
# set the seed
manual_seed = 572
torch.manual_seed(manual_seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

### Project: Yelp review rating prediction

Build a neural network to predict the ratings (1star, 2star, ..., 5star) based on the reviews collected from Yelp.

### Part 1. Preprocessing

In [12]:
import string
import spacy
# from spacy.lang.en.stop_words import STOP_WORDS

# Create our list of punctuationmarks
punctuations = string.punctuation
# Create our list of stop words
nlp = spacy.load('en_core_wOP_WORDS

# Creating our tokenzer function
def tokenize_en(sentence):
    """This function will accepts a sentence as input and processes the sentence into tokens, performing lemmatizing, 
    lowercasing, removing punctuations."""

    tokens = nlp(sentence)
    
    # lemmatizing each token and converting each tons lkike me, I etc
    tokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens]
    
    # d not in punctuations]
    tokens = [word for word in tokens if word not in punctuations]
    
    # Return preprocessed list of tokens
    return tokens 

In [13]:
# load the pretrained word vectors (ref: https://torchtext.readthedocs.io/en/latest/vocab.html#vectors)
from torchtext.vocab import Vectors
vectors = Vectors(name='glove.42B.300d.txt', cache='./drive/My Drive/yelp_review/') # glove.42B.300d.txt is the path to pretrained word embeddings and cache is a directory for caching word vectors (set it to an empty folder)

100%|█████████▉| 1916734/1917494 [03:23<00:00, 10195.73it/s]

In [64]:
# define the TorchText's fields
TEXT = Field(sequential=True, tokenize=tokenize_en, lower=True)
LABEL = Field(sequential=False, unk_token=None)

# Read train, val sets and create iterations
train, val = TabularDataset.splits(
    path="./data/yelp_review/",  # the root directory where the data lies
    train="train.tsv",
    validation="val.tsv",
    format="tsv",
    skip_header=True,
    fields=[("content", TEXT), ("rating", LABEL)],
)

# builds vocabulary using the pre-trained word vectors and all the words that occur at least twice in the training set
TEXT.build_vocab(train, min_freq=3, vectors=vectors)
LABEL.build_vocab(train)

train_iter, val_iter = BucketIterator.splits(
    (train, val),  # we pass in the datasets we want the iterator to draw data from
    batch_sizes=(4, 64),
    sort_key=lambda x: len(x.content),
    sort=True,
    # A key to use for sorting examples in order to batch together examples with similar lengths and minimize padding.
    sort_within_batch=True,
)
print("Preprocessing train and val sets finished")

# Read test set and create test iteration separately due to absence of rating column
test = TabularDataset(
    path="./data/yelp_review/test.tsv",  # absolute path of the test file
    format="tsv",  # format of the test file
    skip_header=True,  # skip the header
    fields=[("content", TEXT)],
)  # column names, note that rating is omitted

test_iter = Iterator(
    dataset=test,  # we pass in the datasets we want the iterator to draw data from
    sort=False,  # don't sort the examples
    batch_size=64,  # batch size
    sort_key=None,  # no sorting
    shuffle=False,  # don't shuffle the examples
    sort_within_batch=False,  # no sorting
    device=device,  # cpu vs cuda
    train=False,  # this is not a train set
)
print("Preprocessing test set finished")

VOCAB_SIZE = len(TEXT.vocab.stoi)

WORD_VEC_SIZE = 300
print(f"VOCAB_SIZE={VOCAB_SIZE}, WORD_VEC_SIZE={WORD_VEC_SIZE}")

Preprocessing train and val sets finished
Preprocessing test set finished
VOCAB_SIZE=14717, WORD_VEC_SIZE=300


In [70]:
# Create a single batch and Test preprocessing on train set
for batch in train_iter:
    reviews = batch.content
    labels = batch.rating
    break  #we use first batch as an example.

print(reviews)
    
# print the four examples with padding and corresponding label
print("processed reviews: ")
for j in range(reviews.shape[1]): # sample loop
    tokens = []
    for i in range(reviews.shape[0]): # token loop
        tokens.append(TEXT.vocab.itos[reviews[i,j]])
    print(j," sample:",tokens," label:", labels[j].item())

tensor([[1139,  105, 1731,  398],
        [   7,   62,  380,    1],
        [   3,  111,  541,    1],
        [ 510,    0,   23,    1],
        [3387,    1,    1,    1]])
processed reviews: 
0  sample: ['hell', 'to', 'the', 'yes', 'adobada']  label: 0
1  sample: ['love', 'their', 'fry', '<unk>', '<pad>']  label: 4
2  sample: ['pricy', 'average', 'korean', 'food', '<pad>']  label: 2
3  sample: ['name', '<pad>', '<pad>', '<pad>', '<pad>']  label: 3


In [76]:
# Test preprocessing on test set
for batch in test_iter:
    reviews = batch.content
    break  #we use first batch as an example.

print(reviews)
    
# print the four examples with padding and corresponding label
print("processed reviews: ")
for j in range(reviews.shape[1]): # sample loop
    tokens = []
    for i in range(reviews.shape[0]): # token loop
        tokens.append(TEXT.vocab.itos[reviews[i,j]])
    print(j," sample:",tokens)

tensor([[  0, 155,   5,  ...,   5,  19,  25],
        [109, 426, 231,  ...,   2,  29, 260],
        [  3,   2,  27,  ..., 533, 795,  23],
        ...,
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1]], device='cuda:0')
processed reviews: 
0  sample: ['<unk>', 'than', 'the', 'average', 'take', 'out', 'but', 'not', '<unk>', 'sit', 'down', 'level', 'it', 'be', 'basically', '<unk>', 'take', 'over', '<unk>', 'this', 'would', 'be', 'it', 'there', 'be', 'certain', '<unk>', 'to', 'its', 'menu', 'give', 'that', 'it', 'be', 'a', 'chain', 'but', 'taste', 'and', 'seasoning', 'deviate', 'base', 'on', 'who', 'be', 'in', 'the', 'kitchen', 'have', 'a', 'half', 'hour', 'break', 'in', 'chinatown', 'on', 'errand', 'so', 'stop', 'in', 'for', 'a', 'quick', 'spicy', 'beef', 'noodle', 'dish', 'the', 'noodle', 'be', 'nice', 'and', 'chewy', 'either', 'someone', 'or', 'the', 'machine', 'stretch', 'the', 'heck', 'out', 'of', 

### Part 2-1. Build model

In [72]:
### Building model
class LSTMmodel(nn.Module):
    def __init__(
        self,
        embedding_size,
        vocab_size,
        output_size,
        hidden_size,
        num_layers,
        nonlin,
        dropout,
    ):
        # In the constructor we define the layers for our model (same as our previous RNN)
        super(LSTMmodel, self).__init__()
        # initialize word embedding lookup table with pretrained word embeddings
        self.embedding = nn.Embedding(
            num_embeddings=vocab_size, embedding_dim=embedding_size
        ).from_pretrained(TEXT.vocab.vectors, freeze=False)
        # self.embedding.weight.data.normal_(0.0,0.05) # mean=0.0, mu=0.05

        # core LSTM module
        self.lstm_rnn_1 = nn.LSTM(
            input_size=embedding_size, hidden_size=hidden_size, num_layers=num_layers
        )  # input_size, hidden_size, num_layers
        self.activation_fn = nonlin
        self.dropout = nn.Dropout(p=dropout)
        self.num_layers = num_layers

        self.linear_layer = nn.Linear(hidden_size, output_size)
        self.softmax_layer = nn.LogSoftmax(dim=-1)

    def forward(self, x):
        # print("x shape:", x.shape)
        # input: (L, N); output: (N, L, H)
        out = self.embedding(x.permute(1, 0))

        # print("embedding output shape:", out.size())
        out = out.permute(1, 0, 2)

        # input: (L, N, H); output: (L, N, num_directions*H)
        out, (h_state, c_state) = self.lstm_rnn_1(out)
        out = self.activation_fn(out.permute(1, 0, 2))
        out = self.dropout(out).permute(1, 0, 2)

        # print("loop output shape:", out.size())
        out = out[-1]  # Pass the last hidden of last layer into linear layer

        out = self.linear_layer(out)
        # print("linear output shape:", out.size())
        out = self.softmax_layer(out)  # accepts 2D or more dimensional inputs
        # print("softmax output shape:", out.size())
        return out

In [19]:
# hyperparameters
MAX_EPOCHS = 5 # number of passes over the training data
LEARNING_RATE = 0.001 # learning rate for the weight update rule
NUM_CLASSES = 5 # number of classes for the problem
EMBEDDING_SIZE = 300 # size of the word embedding
NUM_LAYERS = 1
HIDDEN_SIZE = 500
DROPOUT = 0.5
WEIGHT_DECAY = 0.000001

model = LSTMmodel(EMBEDDING_SIZE, VOCAB_SIZE, NUM_CLASSES, HIDDEN_SIZE, NUM_LAYERS, nn.Tanh(), DROPOUT)
model = model.to(device)
print(model)

LSTMmodel(
  (embedding): Embedding(14717, 300)
  (lstm_rnn_1): LSTM(300, 500)
  (activation_fn): Tanh()
  (dropout): Dropout(p=0.5, inplace=False)
  (linear_layer): Linear(in_features=500, out_features=5, bias=True)
  (softmax_layer): LogSoftmax(dim=-1)
)


In [66]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

def train(loader,model,criterion,optimizer,device):
    total_loss = 0.0
    # iterate throught the data loader
    num_sample = 0
    for batch in loader:
        # load the current batch
        batch_input = batch.content
        batch_output = batch.rating
        
        batch_input = batch_input.to(device)
        batch_output = batch_output.to(device)
        # forward propagation
        # pass the data through the model
        model_outputs = model(batch_input)
        # compute the loss
        cur_loss = criterion(model_outputs, batch_output)
        # total_loss += cur_loss.cpu().item()
        total_loss += cur_loss.item()

        # backward propagation (compute the gradients and update the model)
        # clear the buffer
        optimizer.zero_grad()
        # compute the gradients
        cur_loss.backward()
        # update the weights
        optimizer.step()

        num_sample += batch_output.shape[0]

    return total_loss/num_sample

# evaluation logic based on classification accuracy
def evaluate(loader,model,criterion,device):
    all_pred=[]
    all_label = []
    with torch.no_grad(): # impacts the autograd engine and deactivate it. reduces memory usage and speeds up computation
        for batch in loader:
             # load the current batch
            batch_input = batch.content
            batch_output = batch.rating

            batch_input = batch_input.to(device)
            # forward propagation
            # pass the data through the model
            model_outputs = model(batch_input)
            # identify the predicted class for each example in the batch
            probabilities, predicted = torch.max(model_outputs.cpu().data, 1)
            # put all the true labels and predictions to two lists
            all_pred.extend(predicted)
            # all_label.extend(batch_output.cpu())
            all_label.extend(batch_output)
            
    accuracy = accuracy_score(all_label, all_pred)
    f1score = f1_score(all_label, all_pred, average='macro') 
    return accuracy,f1score

In [85]:
import warnings
warnings.filterwarnings('ignore')

criterion = nn.NLLLoss()
# create an instance of SGD with required hyperparameters
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

# start the training
for epoch in range(MAX_EPOCHS):
    best_f1 = 0
    # train the model for one pass over the data
    train_loss = train(train_iter)  
    # compute the training accuracy
    train_acc, train_f1 = evaluate(train_iter)
    # compute the validation accuracy
    val_acc, val_f1 = evaluate(val_iter)
    
    # print the loss for every epoch
    print('Epoch [{}/{}], Loss: {:.4f}, Training Accuracy: {:.4f}, Training F1: {:.4f}, Validation Accuracy: {:.4f}, Validation F1: {:.4f}'.format(epoch+1, MAX_EPOCHS, train_loss, train_acc, train_f1, val_acc, val_f1))

Epoch [1/5], Loss: 0.2574, Training Accuracy: 0.6349, Training F1: 0.6288, Validation Accuracy: 0.5603, Validation F1: 0.5494
Epoch [2/5], Loss: 0.1958, Training Accuracy: 0.6991, Training F1: 0.6918, Validation Accuracy: 0.5526, Validation F1: 0.5393
Epoch [3/5], Loss: 0.1527, Training Accuracy: 0.7786, Training F1: 0.7743, Validation Accuracy: 0.5549, Validation F1: 0.5450
Epoch [4/5], Loss: 0.1135, Training Accuracy: 0.8520, Training F1: 0.8509, Validation Accuracy: 0.5691, Validation F1: 0.5649
Epoch [5/5], Loss: 0.0848, Training Accuracy: 0.8865, Training F1: 0.8860, Validation Accuracy: 0.5483, Validation F1: 0.5455


### Part 2-2. Hyperparameter tuning

In [24]:
### Random Search
import scipy.stats
import numpy as np


MAX_EPOCHS=10 # can be higher

np.random.seed(manual_seed)  #Note sets for both Numpy and Scipy 
torch.manual_seed(manual_seed) 

def random_search(num_iter):    
    results = []
    for i in range(num_iter):
        config = {
            #define hyperparameters here
            "num_layers": np.random.choice([1, 2]),
            "lr": scipy.stats.loguniform.rvs(10**-4,10**-2),
            "weight_decay": scipy.stats.uniform.rvs(0, 10**-6),
            "hidden_size": scipy.stats.randint(low=200,high=600).rvs(),
            "dropout": np.random.choice([.1, .5, .9]),
            "nonlin" : np.random.choice([nn.ReLU(),nn.Tanh()]),
        }
        
        print("new config - iteration #{}".format(i))
        print(config)
        model = LSTMmodel(EMBEDDING_SIZE, VOCAB_SIZE, NUM_CLASSES, config["hidden_size"], config["num_layers"], config["nonlin"], config["dropout"])
        model.to(device)
        criterion = nn.NLLLoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=config["lr"], weight_decay=config["weight_decay"])

    
        max_f1 = 0
        best_epoch = 0
        for epoch in range(MAX_EPOCHS):
        # train the model for one pass over the data
            train_loss = train(train_iter,model,criterion,optimizer,device)  
        # compute the training accuracy
            train_acc, train_f1 = evaluate(train_iter,model,criterion,device)
        # compute the validation accuracy
            val_acc, val_f1 = evaluate(val_iter,model,criterion,device)
            if val_f1 > max_f1:
                max_f1 = val_f1
                best_epoch = epoch+1

                # save model, optimizer, and number of epoch to a dictionary
                model_save = {
                    'epoch': epoch,  # number of epoch
                    'model_state_dict': model.state_dict(),  # model parameters 
                    'optimizer_state_dict': optimizer.state_dict(),  # save optimizer 
                    'loss': train_loss  # training loss
                }
                # save model 
                torch.save(model_save, "./ckpt/best_model_iter_{}.pt".format(i))
        # print the loss for every epoch
            print('Epoch [{}/{}], Loss: {:.4f}, Training Accuracy: {:.4f}, Training F1: {:.4f}, Validation Accuracy: {:.4f}, Validation F1: {:.4f}'.format(epoch+1, MAX_EPOCHS, train_loss, train_acc, train_f1, val_acc, val_f1))
        results.append((max_f1,best_epoch,config))
        print("Best validation score for iterations #{}: {}".format(i,max_f1))
    return results

In [25]:
random_search(5)

new config - iteration #0
{'num_layers': 2, 'lr': 0.00022226629135794172, 'weight_decay': 6.954413372869395e-07, 'hidden_size': 204, 'dropout': 0.5, 'nonlin': ReLU()}
Epoch [1/10], Loss: 0.4023, Training Accuracy: 0.1994, Training F1: 0.1665, Validation Accuracy: 0.1977, Validation F1: 0.1704
Epoch [2/10], Loss: 0.4023, Training Accuracy: 0.2023, Training F1: 0.1689, Validation Accuracy: 0.1966, Validation F1: 0.1625
Epoch [3/10], Loss: 0.4022, Training Accuracy: 0.2081, Training F1: 0.1707, Validation Accuracy: 0.2000, Validation F1: 0.1585
Epoch [4/10], Loss: 0.4021, Training Accuracy: 0.2032, Training F1: 0.1643, Validation Accuracy: 0.1949, Validation F1: 0.1504
Epoch [5/10], Loss: 0.4021, Training Accuracy: 0.2083, Training F1: 0.1687, Validation Accuracy: 0.1829, Validation F1: 0.1414
Epoch [6/10], Loss: 0.4021, Training Accuracy: 0.2080, Training F1: 0.1677, Validation Accuracy: 0.2083, Validation F1: 0.1630
Epoch [7/10], Loss: 0.4021, Training Accuracy: 0.2105, Training F1: 0.1

[(0.18286621819701673,
  10,
  {'dropout': 0.5,
   'hidden_size': 204,
   'lr': 0.00022226629135794172,
   'nonlin': ReLU(),
   'num_layers': 2,
   'weight_decay': 6.954413372869395e-07}),
 (0.15147337851197815,
  10,
  {'dropout': 0.1,
   'hidden_size': 365,
   'lr': 0.0001237214696506816,
   'nonlin': ReLU(),
   'num_layers': 2,
   'weight_decay': 6.736410214343777e-07}),
 (0.5435842844271062,
  10,
  {'dropout': 0.1,
   'hidden_size': 258,
   'lr': 0.008500484598843613,
   'nonlin': ReLU(),
   'num_layers': 2,
   'weight_decay': 8.885714409599356e-07}),
 (0.46428876398370866,
  10,
  {'dropout': 0.5,
   'hidden_size': 373,
   'lr': 0.004439626674240117,
   'nonlin': ReLU(),
   'num_layers': 2,
   'weight_decay': 8.797617416541492e-07}),
 (0.4689645090031047,
  9,
  {'dropout': 0.5,
   'hidden_size': 224,
   'lr': 0.003151706705674308,
   'nonlin': Tanh(),
   'num_layers': 2,
   'weight_decay': 3.6959843576744966e-07})]

new config - iteration #0  
{'num_layers': 1, 'lr': 0.0010734040518561735, 'weight_decay': 6.193308401435993e-07, 'hidden_size': 384, 'dropout': 0.5, 'nonlin': Tanh()}  
Epoch [1/10], Loss: 0.3998, Training Accuracy: 0.2268, Training F1: 0.1826, Validation Accuracy: 0.2020, Validation F1: 0.1076  
Epoch [2/10], Loss: 0.3977, Training Accuracy: 0.2487, Training F1: 0.2130, Validation Accuracy: 0.2089, Validation F1: 0.1323  
Epoch [3/10], Loss: 0.3943, Training Accuracy: 0.2612, Training F1: 0.2214, Validation Accuracy: 0.2223, Validation F1: 0.1583  
Epoch [4/10], Loss: 0.3896, Training Accuracy: 0.2659, Training F1: 0.2181, Validation Accuracy: 0.2214, Validation F1: 0.1546  
Epoch [5/10], Loss: 0.3770, Training Accuracy: 0.3590, Training F1: 0.3353, Validation Accuracy: 0.3283, Validation F1: 0.2868  
Epoch [6/10], Loss: 0.3558, Training Accuracy: 0.2605, Training F1: 0.2065, Validation Accuracy: 0.2200, Validation F1: 0.1286  
Epoch [7/10], Loss: 0.3718, Training Accuracy: 0.2165, Training F1: 0.1040, Validation Accuracy: 0.2037, Validation F1: 0.0849  
Epoch [8/10], Loss: 0.3339, Training Accuracy: 0.4260, Training F1: 0.4026, Validation Accuracy: 0.3869, Validation F1: 0.3449  
Epoch [9/10], Loss: 0.3318, Training Accuracy: 0.2223, Training F1: 0.1278, Validation Accuracy: 0.2006, Validation F1: 0.0935  
Epoch [10/10], Loss: 0.3789, Training Accuracy: 0.3950, Training F1: 0.3825, Validation Accuracy: 0.3249, Validation F1: 0.2967  
Best validation score for iterations #0: 0.34486946962601916  
new config - iteration #1  
{'num_layers': 1, 'lr': 0.006433366879373084, 'weight_decay': 9.015114625005704e-07, 'hidden_size': 370, 'dropout': 0.1, 'nonlin': Tanh()}  
Epoch [1/10], Loss: 0.3891, Training Accuracy: 0.3268, Training F1: 0.2849, Validation Accuracy: 0.2663, Validation F1: 0.1963  
Epoch [2/10], Loss: 0.3585, Training Accuracy: 0.2706, Training F1: 0.2060, Validation Accuracy: 0.2189, Validation F1: 0.1154  
Epoch [3/10], Loss: 0.3006, Training Accuracy: 0.4815, Training F1: 0.4634, Validation Accuracy: 0.4389, Validation F1: 0.4025  
Epoch [4/10], Loss: 0.2752, Training Accuracy: 0.4873, Training F1: 0.4732, Validation Accuracy: 0.4529, Validation F1: 0.4282  
Epoch [5/10], Loss: 0.2612, Training Accuracy: 0.5081, Training F1: 0.5090, Validation Accuracy: 0.4809, Validation F1: 0.4776  
Epoch [6/10], Loss: 0.2509, Training Accuracy: 0.5502, Training F1: 0.5505, Validation Accuracy: 0.5174, Validation F1: 0.5121  
Epoch [7/10], Loss: 0.2433, Training Accuracy: 0.5507, Training F1: 0.5512, Validation Accuracy: 0.5203, Validation F1: 0.5141  
Epoch [8/10], Loss: 0.2355, Training Accuracy: 0.5363, Training F1: 0.5389, Validation Accuracy: 0.4937, Validation F1: 0.4958  
Epoch [9/10], Loss: 0.2298, Training Accuracy: 0.5612, Training F1: 0.5662, Validation Accuracy: 0.5123, Validation F1: 0.5154  
Epoch [10/10], Loss: 0.2252, Training Accuracy: 0.5647, Training F1: 0.5689, Validation Accuracy: 0.5177, Validation F1: 0.5214  
Best validation score for iterations #1: 0.5214360507127358  
new config - iteration #2  
{'num_layers': 2, 'lr': 0.005517780183436344, 'weight_decay': 7.492728978678677e-07, 'hidden_size': 396, 'dropout': 0.5, 'nonlin': ReLU()}  
Epoch [1/10], Loss: 0.4001, Training Accuracy: 0.2119, Training F1: 0.1145, Validation Accuracy: 0.1963, Validation F1: 0.1127  
Epoch [2/10], Loss: 0.3967, Training Accuracy: 0.2290, Training F1: 0.1333, Validation Accuracy: 0.2169, Validation F1: 0.1228  
Epoch [3/10], Loss: 0.3856, Training Accuracy: 0.2016, Training F1: 0.0760, Validation Accuracy: 0.1954, Validation F1: 0.0692  
Epoch [4/10], Loss: 0.3778, Training Accuracy: 0.2217, Training F1: 0.1153, Validation Accuracy: 0.2094, Validation F1: 0.0997  
Epoch [5/10], Loss: 0.3596, Training Accuracy: 0.2785, Training F1: 0.2105, Validation Accuracy: 0.2383, Validation F1: 0.1481  
Epoch [6/10], Loss: 0.3515, Training Accuracy: 0.3246, Training F1: 0.2722, Validation Accuracy: 0.2720, Validation F1: 0.2000  
Epoch [7/10], Loss: 0.3221, Training Accuracy: 0.3153, Training F1: 0.2531, Validation Accuracy: 0.2803, Validation F1: 0.2122  
Epoch [8/10], Loss: 0.2980, Training Accuracy: 0.3737, Training F1: 0.3248, Validation Accuracy: 0.3343, Validation F1: 0.2768  
Epoch [9/10], Loss: 0.2832, Training Accuracy: 0.4888, Training F1: 0.4749, Validation Accuracy: 0.4617, Validation F1: 0.4421  
Epoch [10/10], Loss: 0.2690, Training Accuracy: 0.5024, Training F1: 0.4782, Validation Accuracy: 0.4854, Validation F1: 0.4551  
Best validation score for iterations #2: 0.4550926856284935  
new config - iteration #3  
{'num_layers': 1, 'lr': 0.0026897169642907844, 'weight_decay': 9.79722991168056e-07, 'hidden_size': 486, 'dropout': 0.9, 'nonlin': ReLU()}  
Epoch [1/10], Loss: 0.4003, Training Accuracy: 0.2158, Training F1: 0.1760, Validation Accuracy: 0.1917, Validation F1: 0.1477  
Epoch [2/10], Loss: 0.3980, Training Accuracy: 0.2320, Training F1: 0.1887, Validation Accuracy: 0.2260, Validation F1: 0.1702  
Epoch [3/10], Loss: 0.3920, Training Accuracy: 0.2535, Training F1: 0.2036, Validation Accuracy: 0.2171, Validation F1: 0.1484  
Epoch [4/10], Loss: 0.3734, Training Accuracy: 0.2442, Training F1: 0.2017, Validation Accuracy: 0.2171, Validation F1: 0.1482  
Epoch [5/10], Loss: 0.3580, Training Accuracy: 0.2096, Training F1: 0.1874, Validation Accuracy: 0.1974, Validation F1: 0.1644  
Epoch [6/10], Loss: 0.3995, Training Accuracy: 0.2214, Training F1: 0.1560, Validation Accuracy: 0.2017, Validation F1: 0.1214  
Epoch [7/10], Loss: 0.3909, Training Accuracy: 0.2371, Training F1: 0.1783, Validation Accuracy: 0.2009, Validation F1: 0.1253  
Epoch [8/10], Loss: 0.3727, Training Accuracy: 0.2206, Training F1: 0.1394, Validation Accuracy: 0.2057, Validation F1: 0.1085  
Epoch [9/10], Loss: 0.3672, Training Accuracy: 0.2379, Training F1: 0.1864, Validation Accuracy: 0.2103, Validation F1: 0.1289  
Epoch [10/10], Loss: 0.3785, Training Accuracy: 0.2360, Training F1: 0.1529, Validation Accuracy: 0.2166, Validation F1: 0.1211  
Best validation score for iterations #3: 0.1702405082539765  
new config - iteration #4  
{'num_layers': 1, 'lr': 0.0003128920152156375, 'weight_decay': 5.535718450103309e-07, 'hidden_size': 413, 'dropout': 0.5, 'nonlin': ReLU()}  
Epoch [1/10], Loss: 0.4024, Training Accuracy: 0.2046, Training F1: 0.1637, Validation Accuracy: 0.2043, Validation F1: 0.1353  
Epoch [2/10], Loss: 0.4020, Training Accuracy: 0.2104, Training F1: 0.1762, Validation Accuracy: 0.2009, Validation F1: 0.1287  
Epoch [3/10], Loss: 0.4016, Training Accuracy: 0.2183, Training F1: 0.1852, Validation Accuracy: 0.2034, Validation F1: 0.1366  
Epoch [4/10], Loss: 0.4013, Training Accuracy: 0.2244, Training F1: 0.1939, Validation Accuracy: 0.2140, Validation F1: 0.1429  
Epoch [5/10], Loss: 0.4010, Training Accuracy: 0.2305, Training F1: 0.2018, Validation Accuracy: 0.2183, Validation F1: 0.1569  
Epoch [6/10], Loss: 0.4006, Training Accuracy: 0.2309, Training F1: 0.2024, Validation Accuracy: 0.2146, Validation F1: 0.1539  
Epoch [7/10], Loss: 0.4002, Training Accuracy: 0.2394, Training F1: 0.2134, Validation Accuracy: 0.2126, Validation F1: 0.1605  
Epoch [8/10], Loss: 0.3998, Training Accuracy: 0.2476, Training F1: 0.2221, Validation Accuracy: 0.2126, Validation F1: 0.1665  
Epoch [9/10], Loss: 0.3993, Training Accuracy: 0.2446, Training F1: 0.2222, Validation Accuracy: 0.2149, Validation F1: 0.1722  
Epoch [10/10], Loss: 0.3987, Training Accuracy: 0.2579, Training F1: 0.2345, Validation Accuracy: 0.2277, Validation F1: 0.1875  
Best validation score for iterations #4: 0.18748706624102152  
new config - iteration #5  
{'num_layers': 1, 'lr': 0.008238061289204134, 'weight_decay': 9.17001957380131e-07, 'hidden_size': 362, 'dropout': 0.9, 'nonlin': Tanh()}  
Epoch [1/10], Loss: 0.3926, Training Accuracy: 0.3252, Training F1: 0.3246, Validation Accuracy: 0.3189, Validation F1: 0.3093  
Epoch [2/10], Loss: 0.3667, Training Accuracy: 0.3596, Training F1: 0.3365, Validation Accuracy: 0.3354, Validation F1: 0.2946  
Epoch [3/10], Loss: 0.3292, Training Accuracy: 0.4181, Training F1: 0.4168, Validation Accuracy: 0.4169, Validation F1: 0.4009  
Epoch [4/10], Loss: 0.2959, Training Accuracy: 0.4644, Training F1: 0.4579, Validation Accuracy: 0.4540, Validation F1: 0.4372  
Epoch [5/10], Loss: 0.2788, Training Accuracy: 0.4736, Training F1: 0.4794, Validation Accuracy: 0.4743, Validation F1: 0.4744  
Epoch [6/10], Loss: 0.2683, Training Accuracy: 0.4856, Training F1: 0.4926, Validation Accuracy: 0.4900, Validation F1: 0.4938  
Epoch [7/10], Loss: 0.2591, Training Accuracy: 0.4783, Training F1: 0.4822, Validation Accuracy: 0.4854, Validation F1: 0.4885  
Epoch [8/10], Loss: 0.2523, Training Accuracy: 0.5149, Training F1: 0.5199, Validation Accuracy: 0.5026, Validation F1: 0.5076  
Epoch [9/10], Loss: 0.2465, Training Accuracy: 0.5172, Training F1: 0.5204, Validation Accuracy: 0.5029, Validation F1: 0.5066  
Epoch [10/10], Loss: 0.2419, Training Accuracy: 0.5009, Training F1: 0.5035, Validation Accuracy: 0.4920, Validation F1: 0.4952  
Best validation score for iterations #5: 0.5076457999862197  
new config - iteration #6  
{'num_layers': 2, 'lr': 0.004416036055274488, 'weight_decay': 3.860062802155517e-07, 'hidden_size': 321, 'dropout': 0.9, 'nonlin': Tanh()}  
Epoch [1/10], Loss: 0.4003, Training Accuracy: 0.2035, Training F1: 0.1286, Validation Accuracy: 0.1977, Validation F1: 0.1262  
Epoch [2/10], Loss: 0.3973, Training Accuracy: 0.2221, Training F1: 0.1394, Validation Accuracy: 0.2123, Validation F1: 0.1433  
Epoch [3/10], Loss: 0.3776, Training Accuracy: 0.2147, Training F1: 0.1183, Validation Accuracy: 0.2060, Validation F1: 0.1196  
Epoch [4/10], Loss: 0.3588, Training Accuracy: 0.3845, Training F1: 0.3768, Validation Accuracy: 0.3589, Validation F1: 0.3464  
Epoch [5/10], Loss: 0.3091, Training Accuracy: 0.4280, Training F1: 0.4205, Validation Accuracy: 0.4100, Validation F1: 0.3979  
Epoch [6/10], Loss: 0.2868, Training Accuracy: 0.4694, Training F1: 0.4660, Validation Accuracy: 0.4589, Validation F1: 0.4474  
Epoch [7/10], Loss: 0.2756, Training Accuracy: 0.4735, Training F1: 0.4780, Validation Accuracy: 0.4491, Validation F1: 0.4490  
Epoch [8/10], Loss: 0.2676, Training Accuracy: 0.4829, Training F1: 0.4891, Validation Accuracy: 0.4634, Validation F1: 0.4654  
Epoch [9/10], Loss: 0.2583, Training Accuracy: 0.5060, Training F1: 0.5123, Validation Accuracy: 0.4900, Validation F1: 0.4927  
Epoch [10/10], Loss: 0.2519, Training Accuracy: 0.4994, Training F1: 0.5038, Validation Accuracy: 0.4891, Validation F1: 0.4918  
Best validation score for iterations #6: 0.49270761574179583  
new config - iteration #7  
{'num_layers': 1, 'lr': 0.004220631907754142, 'weight_decay': 8.442937150761966e-07, 'hidden_size': 383, 'dropout': 0.5, 'nonlin': Tanh()}  
Epoch [1/10], Loss: 0.3954, Training Accuracy: 0.2291, Training F1: 0.1437, Validation Accuracy: 0.2097, Validation F1: 0.1142  
Epoch [2/10], Loss: 0.3702, Training Accuracy: 0.2192, Training F1: 0.1181, Validation Accuracy: 0.2017, Validation F1: 0.0837  
Epoch [3/10], Loss: 0.3531, Training Accuracy: 0.3266, Training F1: 0.2866, Validation Accuracy: 0.2477, Validation F1: 0.1731  
Epoch [4/10], Loss: 0.3293, Training Accuracy: 0.4329, Training F1: 0.4227, Validation Accuracy: 0.4046, Validation F1: 0.3797  
Epoch [5/10], Loss: 0.3215, Training Accuracy: 0.3443, Training F1: 0.2938, Validation Accuracy: 0.2694, Validation F1: 0.1978  
Epoch [6/10], Loss: 0.2989, Training Accuracy: 0.4151, Training F1: 0.3650, Validation Accuracy: 0.3666, Validation F1: 0.3110  
Epoch [7/10], Loss: 0.2832, Training Accuracy: 0.5009, Training F1: 0.4977, Validation Accuracy: 0.4709, Validation F1: 0.4530  
Epoch [8/10], Loss: 0.2682, Training Accuracy: 0.5082, Training F1: 0.5105, Validation Accuracy: 0.4957, Validation F1: 0.4896  
Epoch [9/10], Loss: 0.2610, Training Accuracy: 0.5129, Training F1: 0.5109, Validation Accuracy: 0.4954, Validation F1: 0.4866  
Epoch [10/10], Loss: 0.2561, Training Accuracy: 0.4669, Training F1: 0.4535, Validation Accuracy: 0.4271, Validation F1: 0.4094  
Best validation score for iterations #7: 0.48958051972704864  
new config - iteration #8  
{'num_layers': 2, 'lr': 0.00739731557328295, 'weight_decay': 9.176389095848678e-07, 'hidden_size': 251, 'dropout': 0.5, 'nonlin': ReLU()}  
Epoch [1/10], Loss: 0.3997, Training Accuracy: 0.2060, Training F1: 0.0894, Validation Accuracy: 0.1977, Validation F1: 0.0770  
Epoch [2/10], Loss: 0.3904, Training Accuracy: 0.2012, Training F1: 0.0695, Validation Accuracy: 0.1937, Validation F1: 0.0856  
Epoch [3/10], Loss: 0.3878, Training Accuracy: 0.2945, Training F1: 0.2166, Validation Accuracy: 0.2640, Validation F1: 0.1813  
Epoch [4/10], Loss: 0.3700, Training Accuracy: 0.2004, Training F1: 0.0676, Validation Accuracy: 0.1954, Validation F1: 0.0659  
Epoch [5/10], Loss: 0.4000, Training Accuracy: 0.2008, Training F1: 0.0675, Validation Accuracy: 0.1954, Validation F1: 0.0659  
Epoch [6/10], Loss: 0.3755, Training Accuracy: 0.2821, Training F1: 0.2353, Validation Accuracy: 0.2491, Validation F1: 0.1807  
Epoch [7/10], Loss: 0.3051, Training Accuracy: 0.4154, Training F1: 0.4192, Validation Accuracy: 0.3903, Validation F1: 0.3906  
Epoch [8/10], Loss: 0.2870, Training Accuracy: 0.4227, Training F1: 0.4236, Validation Accuracy: 0.4089, Validation F1: 0.4102  
Epoch [9/10], Loss: 0.2778, Training Accuracy: 0.4293, Training F1: 0.4306, Validation Accuracy: 0.4143, Validation F1: 0.4155  
Epoch [10/10], Loss: 0.2721, Training Accuracy: 0.4319, Training F1: 0.4313, Validation Accuracy: 0.4163, Validation F1: 0.4160  
Best validation score for iterations #8: 0.4160003156041802  
new config - iteration #9  
{'num_layers': 1, 'lr': 0.00020904984546824364, 'weight_decay': 2.9892338992590436e-07, 'hidden_size': 394, 'dropout': 0.5, 'nonlin': Tanh()}  
Epoch [1/10], Loss: 0.4020, Training Accuracy: 0.2131, Training F1: 0.1870, Validation Accuracy: 0.2134, Validation F1: 0.1626  
Epoch [2/10], Loss: 0.4016, Training Accuracy: 0.2088, Training F1: 0.1815, Validation Accuracy: 0.2031, Validation F1: 0.1586  
Epoch [3/10], Loss: 0.4013, Training Accuracy: 0.2200, Training F1: 0.1924, Validation Accuracy: 0.2003, Validation F1: 0.1587  
Epoch [4/10], Loss: 0.4007, Training Accuracy: 0.2269, Training F1: 0.1999, Validation Accuracy: 0.2063, Validation F1: 0.1645  
Epoch [5/10], Loss: 0.4003, Training Accuracy: 0.2331, Training F1: 0.2085, Validation Accuracy: 0.2126, Validation F1: 0.1742  
Epoch [6/10], Loss: 0.3998, Training Accuracy: 0.2382, Training F1: 0.2149, Validation Accuracy: 0.2171, Validation F1: 0.1777  
Epoch [7/10], Loss: 0.3992, Training Accuracy: 0.2425, Training F1: 0.2197, Validation Accuracy: 0.2183, Validation F1: 0.1797  
Epoch [8/10], Loss: 0.3986, Training Accuracy: 0.2489, Training F1: 0.2284, Validation Accuracy: 0.2209, Validation F1: 0.1894  
Epoch [9/10], Loss: 0.3981, Training Accuracy: 0.2531, Training F1: 0.2337, Validation Accuracy: 0.2343, Validation F1: 0.2007  
Epoch [10/10], Loss: 0.3975, Training Accuracy: 0.2598, Training F1: 0.2414, Validation Accuracy: 0.2320, Validation F1: 0.1969  
Best validation score for iterations #9: 0.20065014288262914  
new config - iteration #10  
{'num_layers': 1, 'lr': 0.0012461091967769465, 'weight_decay': 1.3911835645776803e-08, 'hidden_size': 369, 'dropout': 0.9, 'nonlin': Tanh()}  
Epoch [1/10], Loss: 0.4016, Training Accuracy: 0.2225, Training F1: 0.1950, Validation Accuracy: 0.2080, Validation F1: 0.1799  
Epoch [2/10], Loss: 0.3988, Training Accuracy: 0.2385, Training F1: 0.2082, Validation Accuracy: 0.2217, Validation F1: 0.1841  
Epoch [3/10], Loss: 0.3959, Training Accuracy: 0.2484, Training F1: 0.2147, Validation Accuracy: 0.2217, Validation F1: 0.1751  
Epoch [4/10], Loss: 0.3911, Training Accuracy: 0.2581, Training F1: 0.2158, Validation Accuracy: 0.2160, Validation F1: 0.1532  
Epoch [5/10], Loss: 0.3756, Training Accuracy: 0.3438, Training F1: 0.3427, Validation Accuracy: 0.3386, Validation F1: 0.3293  
Epoch [6/10], Loss: 0.3585, Training Accuracy: 0.2784, Training F1: 0.2493, Validation Accuracy: 0.2246, Validation F1: 0.1594  
Epoch [7/10], Loss: 0.3659, Training Accuracy: 0.2288, Training F1: 0.1902, Validation Accuracy: 0.2189, Validation F1: 0.1740  
Epoch [8/10], Loss: 0.3868, Training Accuracy: 0.3683, Training F1: 0.3540, Validation Accuracy: 0.3429, Validation F1: 0.3195  
Epoch [9/10], Loss: 0.3425, Training Accuracy: 0.2288, Training F1: 0.1794, Validation Accuracy: 0.2003, Validation F1: 0.1227  
Epoch [10/10], Loss: 0.3926, Training Accuracy: 0.2642, Training F1: 0.2177, Validation Accuracy: 0.2229, Validation F1: 0.1402  
Best validation score for iterations #10: 0.3293307594490423  
new config - iteration #11  
{'num_layers': 1, 'lr': 0.0010123425280416083, 'weight_decay': 7.721490895223433e-07, 'hidden_size': 400, 'dropout': 0.1, 'nonlin': Tanh()}  
Epoch [1/10], Loss: 0.3998, Training Accuracy: 0.2191, Training F1: 0.1448, Validation Accuracy: 0.1991, Validation F1: 0.1011  
Epoch [2/10], Loss: 0.3975, Training Accuracy: 0.2457, Training F1: 0.2031, Validation Accuracy: 0.2163, Validation F1: 0.1466  
Epoch [3/10], Loss: 0.3941, Training Accuracy: 0.2626, Training F1: 0.2226, Validation Accuracy: 0.2220, Validation F1: 0.1470  
Epoch [4/10], Loss: 0.3894, Training Accuracy: 0.2656, Training F1: 0.2152, Validation Accuracy: 0.2223, Validation F1: 0.1356  
Epoch [5/10], Loss: 0.3749, Training Accuracy: 0.3549, Training F1: 0.2972, Validation Accuracy: 0.3351, Validation F1: 0.2769  
Epoch [6/10], Loss: 0.3531, Training Accuracy: 0.3739, Training F1: 0.3609, Validation Accuracy: 0.3117, Validation F1: 0.2793  
Epoch [7/10], Loss: 0.3413, Training Accuracy: 0.3046, Training F1: 0.2588, Validation Accuracy: 0.2429, Validation F1: 0.1632  
Epoch [8/10], Loss: 0.3219, Training Accuracy: 0.3750, Training F1: 0.3452, Validation Accuracy: 0.3629, Validation F1: 0.2982  
Epoch [9/10], Loss: 0.3168, Training Accuracy: 0.2501, Training F1: 0.1683, Validation Accuracy: 0.2234, Validation F1: 0.1244  
Epoch [10/10], Loss: 0.3197, Training Accuracy: 0.4007, Training F1: 0.3805, Validation Accuracy: 0.2894, Validation F1: 0.2375  
Best validation score for iterations #11: 0.29818053371670883  


### Part 3. Predicting on test set

In [74]:
# define a new model
# model2 = LSTMmodel(EMBEDDING_SIZE, VOCAB_SIZE, NUM_CLASSES, HIDDEN_SIZE, NUM_LAYERS, nn.Tanh(), DROPOUT) 
model2 = LSTMmodel(EMBEDDING_SIZE, VOCAB_SIZE, NUM_CLASSES, 258, 2, nn.ReLU(), 0.1) 

# load checkpoint 
checkpoint = torch.load("./ckpt/best_model_iter_2.pt")

# assign the parameters of checkpoint to this new model
model2.load_state_dict(checkpoint['model_state_dict'])
model2.to(device)

print(model2) # can be used for inference or for further training

LSTMmodel(
  (embedding): Embedding(14717, 300)
  (lstm_rnn_1): LSTM(300, 258, num_layers=2)
  (activation_fn): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (linear_layer): Linear(in_features=258, out_features=5, bias=True)
  (softmax_layer): LogSoftmax(dim=-1)
)


In [58]:
# Convert the predicted ids to class labels
def create_test_predictions(loader, model):
    prediction_list=[]
    with torch.no_grad(): # impacts the autograd engine and deactivate it. reduces memory usage and speeds up computation
        for batch in loader:
            # load the current batch
            batch_input = batch.content
            batch_input = batch_input.to(device)
            # forward propagation
            # pass the data through the model
            model_outputs = model(batch_input)
            # identify the predicted class for each example in the batch
            probabilities, predicted = torch.max(model_outputs.cpu().data, 1)
            # put all the predictions to prediction_list
            for example_prediction in predicted:
                prediction_list.append(LABEL.vocab.itos[example_prediction.cpu().item()])
    return prediction_list
prediction_list = create_test_predictions(test_iter, model2)

In [ ]:
# funtion for save prediction
def out_prediction(date, prediction_list):
    """
    out_prediction takes two input varibles: date, prediction_list
    <date>, string, date
    <prediction_list>, list of string which includes all your predications of TEST samples
                        e.g., ['1star','5star','3star']
                        
    Generate a file is named with SYS_PRED_<date>.txt in current directory
    """
    output_file = open("SYS_PRED_{}.txt".format(date),'w')
    for item in prediction_list:
        output_file.write(item+"\n")
    output_file.close()   

In [60]:
# Write predictions into file
out_prediction(2021/01/16n', prediction_list)